In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torchdp import PrivacyEngine

In [5]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/guoxinli/.pyenv/versions/3.7.0/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [10]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 1
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False
        self.sigma = 1.0
        self.max_per_sample_grad_norm = 1
        self.microbatches = 1
        self.norm_clip = 1
        self.noise_multiplier = 1
        
args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [7]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [8]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [9]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [13]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment
# optimizer = DPSGD(model.parameters(),lr=args.lr,batch_size=args.batch_size//args.microbatches,C=args.norm_clip,noise_multiplier=args.noise_multiplier)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.298883
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.185310
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.934312
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.385879
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.805123
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.718132
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.510358
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.523555
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.462892
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.416218
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.250322
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.250731
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.248141
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.272166
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.228075
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.186283
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.203871
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.270535
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.204378
Train Epoch: 1 [36480/60032 (61%)]	Loss: 0.410582
Train Epoch:

In [3]:
import math
import torch

import uuid
import torch
from torch.optim.optimizer import Optimizer, required

import torch.nn.functional as F
PI= torch.FloatTensor([math.pi])
NORM = (torch.sqrt(2.0*PI))
def norm_pdf(x,mean,std):
    y= (x-mean)/std
    return (torch.exp( -(y).pow(2)/2.0)/NORM)/std

class DPSGD(Optimizer):
    

    def __init__(self, params, lr=required, momentum=0, dampening=0,
                 weight_decay=0, nesterov=False,C=1,noise_multiplier= 1.0 , batch_size=256):
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))

        defaults = dict(lr=lr, momentum=momentum, dampening=dampening,
                        weight_decay=weight_decay, nesterov=nesterov)
        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")
        super(DPSGD, self).__init__(params, defaults)

        
        self.batch_size = batch_size
        
        self.C = C
        self.bigger_batch = {}


        self.bigger_batch_count = {}
        self.noise_multiplier = noise_multiplier
        
    def __setstate__(self, state):
        super(SGD, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        noise_std =self.noise_multiplier *self.C
        
        loss = None
        if closure is not None:
            loss = closure()

        
        norm  = 0
        
            
        for group in self.param_groups:
        
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                
                
                if not hasattr(p,'myid'):
                    p.myid = uuid.uuid4()
                    self.bigger_batch[p.myid] = torch.zeros_like(grad)
                    
                    self.bigger_batch_count[p.myid] =  torch.cuda.LongTensor(size=[1]).zero_() if self.bigger_batch[p.myid].is_cuda else  torch.LongTensor(size=[1]).zero_()

                norm+=grad.norm()**2.0
        norm=norm**(0.5)
                
        
        for group in self.param_groups:
        
            for p in group['params']:
                grad = p.grad.data
                
                
                cliped = (grad*self.C) /( torch.max(norm,torch.ones_like(norm)*self.C))
                self.bigger_batch [p.myid].add_(cliped)
                self.bigger_batch_count[p.myid]+=1
                

        if   self.bigger_batch_count[p.myid] == self.batch_size:


            for group in self.param_groups:
            
                for p in group['params']:
                    
                    

                    
                    base =  self.bigger_batch[p.myid]
                    my_rand = torch.zeros_like(base) 
                    my_rand.normal_(mean=0, std =noise_std  )
                    
                    base.add_(my_rand)
                
                    base = base/float(self.batch_size) 

                    self.bigger_batch[p.myid] =base
                    

                    
                   
                                    
            
            


            for group in self.param_groups:
                weight_decay = group['weight_decay']
                momentum = group['momentum']
                dampening = group['dampening']
                nesterov = group['nesterov']

                for p in group['params']:
                    if p.grad is None:
                        continue
                    d_p = self.bigger_batch[p.myid]
                    if weight_decay != 0:
                        d_p.add_(weight_decay, p.data)
                    if momentum != 0:
                        param_state = self.state[p]
                        if 'momentum_buffer' not in param_state:
                            buf = param_state['momentum_buffer'] = torch.clone(d_p).detach()
                        else:
                            buf = param_state['momentum_buffer']
                            buf.mul_(momentum).add_(1 - dampening, d_p)
                        if nesterov:
                            d_p = d_p.add(momentum, buf)
                        else:
                            d_p = buf

                    p.data.add_(-group['lr'], d_p)
                    self.bigger_batch[p.myid].zero_()
                    self.bigger_batch_count[p.myid].zero_()

        return loss